In [1]:
#from google.colab import drive
#drive.mount("/content/drive", force_remount=True)

## Import Dependencies

In [1]:
#file
import json
import xml.etree.ElementTree as ET
import os

#data science
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import numpy as np

#images
from PIL import Image
import cv2
import h5py
from skimage import util
from skimage.measure import label, regionprops
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
# from skimage.measure import compare_psnr updated to peak_signal_noise_ratio
# from skimage.measure import compare_ssim updated to structural_similarity

#stats
import math 
import random
import scipy.io as sio
import scipy
from scipy.ndimage import gaussian_filter, morphology

#torch modules
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.optim.lr_scheduler import StepLR, MultiStepLR

#model
from sklearn import linear_model

#time
import argparse
from time import time

## Adjust And Format Image Files

In [2]:
def read_image(x):
    img_arr = np.array(Image.open(x))
    if len(img_arr.shape) == 2:  # grayscale
        img_arr = np.tile(img_arr, [3, 1, 1]).transpose(1, 2, 0)
    return img_arr

In [3]:
class RandomCrop(object):
    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):

        image, target, gtcount = sample['image'], sample['target'], sample['gtcount']
        h, w = image.shape[:2]

        if isinstance(self.output_size, tuple):
            new_h = min(self.output_size[0], h)
            new_w = min(self.output_size[1], w)
            assert (new_h, new_w) == self.output_size
        else:
            crop_size = min(self.output_size, h, w)
            assert crop_size == self.output_size
            new_h = new_w = crop_size
        if gtcount > 0:
            mask = target > 0
            ch, cw = int(np.ceil(new_h / 2)), int(np.ceil(new_w / 2))
            mask_center = np.zeros((h, w), dtype=np.uint8)
            mask_center[ch:h-ch+1, cw:w-cw+1] = 1
            mask = (mask & mask_center)
            idh, idw = np.where(mask == 1)
            if len(idh) != 0:
                ids = random.choice(range(len(idh)))
                hc, wc = idh[ids], idw[ids]
                top, left = hc-ch, wc-cw
            else:
                top = np.random.randint(0, h-new_h+1)
                left = np.random.randint(0, w-new_w+1)
        else:
            top = np.random.randint(0, h-new_h+1)
            left = np.random.randint(0, w-new_w+1)

        image = image[top:top+new_h, left:left+new_w, :]
        target = target[top:top+new_h, left:left+new_w]

        return {'image': image, 'target': target, 'gtcount': gtcount}

In [4]:
class RandomFlip(object):
    def __init__(self):
        pass

    def __call__(self, sample):
        image, target, gtcount = sample['image'], sample['target'], sample['gtcount']
        do_mirror = np.random.randint(2)
        if do_mirror:
            image = cv2.flip(image, 1)
            target = cv2.flip(target, 1)
        return {'image': image, 'target': target, 'gtcount': gtcount}

In [5]:
class Normalize(object):

    def __init__(self, scale, mean, std):
        self.scale = scale
        self.mean = mean
        self.std = std

    def __call__(self, sample):
        image, target, gtcount = sample['image'], sample['target'], sample['gtcount']
        image, target = image.astype('float32'), target.astype('float32')

        # pixel normalization
        image = (self.scale * image - self.mean) / self.std

        image, target = image.astype('float32'), target.astype('float32')

        return {'image': image, 'target': target, 'gtcount': gtcount}

In [6]:
class ZeroPadding(object):
    def __init__(self, psize=32):
        self.psize = psize

    def __call__(self, sample):
        psize =  self.psize

        image, target, gtcount = sample['image'], sample['target'], sample['gtcount']
        h,w = image.size()[-2:]
        ph,pw = (psize-h%psize),(psize-w%psize)
        print(ph,pw)

        (pl, pr) = (pw//2, pw-pw//2) if pw != psize else (0, 0)
        (pt, pb) = (ph//2, ph-ph//2) if ph != psize else (0, 0)
        if (ph!=psize) or (pw!=psize):
            tmp_pad = [pl, pr, pt, pb]
            print(tmp_pad)
            image = F.pad(image,tmp_pad)
            target = F.pad(target,tmp_pad)

        return {'image': image, 'target': target, 'gtcount': gtcount}

In [7]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __init__(self):
        pass

    def __call__(self, sample):
        # swap color axis
        # numpy image: H x W x C
        # torch image: C X H X W
        image, target, gtcount = sample['image'], sample['target'], sample['gtcount']
        image = image.transpose((2, 0, 1))
        target = np.expand_dims(target, axis=2)
        target = target.transpose((2, 0, 1))
        image, target = torch.from_numpy(image), torch.from_numpy(target)
        return {'image': image, 'target': target, 'gtcount': gtcount}

In [8]:
class MaizeTasselDataset(Dataset):
    def __init__(self, data_dir, data_list, ratio, train=True, transform=None):
        self.data_dir = data_dir
        self.data_list = [name.split('\t') for name in open(data_list).read().splitlines()]
        self.ratio = ratio
        self.train = train
        self.transform = transform
        self.image_list = []
        
        # store images and generate ground truths
        self.images = {}
        self.targets = {}
        self.gtcounts = {}
        self.dotimages = {}

    def bbs2points(self, bbs):    
        points = []
        for bb in bbs:
            x1, y1, w, h = [float(b) for b in bb]
            x2, y2 = x1+w-1, y1+h-1
            x, y = np.round((x1+x2)/2).astype(np.int32), np.round((y1+y2)/2).astype(np.int32)
            points.append([x, y])
        return points

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        file_name = self.data_list[idx]
        self.image_list.append(file_name[0])
        if file_name[0] not in self.images:
            image = read_image(self.data_dir+file_name[0])
            annotation = sio.loadmat(self.data_dir+file_name[1])
            h, w = image.shape[:2]
            nh = int(np.ceil(h * self.ratio))
            nw = int(np.ceil(w * self.ratio))
            image = cv2.resize(image, (nw, nh), interpolation = cv2.INTER_CUBIC)
            target = np.zeros((nh, nw), dtype=np.float32)
            dotimage = image.copy()
            if annotation['annotation'][0][0][1] is not None:
                bbs = annotation['annotation'][0][0][1]
                gtcount = bbs.shape[0]
                pts = self.bbs2points(bbs)
                for pt in pts:
                    pt[0], pt[1] = int(pt[0] * self.ratio), int(pt[1] * self.ratio)
                    target[pt[1], pt[0]] = 1
                    cv2.circle(dotimage, (pt[0], pt[1]), int(24 * self.ratio) , (255, 0, 0), -1)
            else:
                gtcount = 0
            target = gaussian_filter(target, 80 * self.ratio)

            #plt.imshow(target, cmap=cm.jet)
            #plt.show()
            print(target.sum())

            self.images.update({file_name[0]:image})
            self.targets.update({file_name[0]:target})
            self.gtcounts.update({file_name[0]:gtcount})
            self.dotimages.update({file_name[0]:dotimage})

        
        sample = {
            'image': self.images[file_name[0]], 
            'target': self.targets[file_name[0]], 
            'gtcount': self.gtcounts[file_name[0]]
        }

        if self.transform:
            sample = self.transform(sample)

        return sample

## Build Data Loader

In [9]:
dataset = MaizeTasselDataset(
        data_dir='maize_counting_dataset', 
        data_list='maize_counting_dataset/train.txt',
        ratio=0.167, 
        train=True, 
        transform=transforms.Compose([
            ToTensor()]
        )
    )

In [10]:
dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=1,
        shuffle=False,
        num_workers=0
    )

In [12]:
print(len(dataloader))
mean = 0.
std = 0.
for i, data in enumerate(dataloader, 0):
        images, targets = data['image'], data['target']
        bs = images.size(0)
        images = images.view(bs, images.size(1), -1).float()
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        print(f"index = {i} size = {images.size()}")
mean /= len(dataloader)
std /= len(dataloader)
print(mean/255.)
print(std/255.)

186
96.999985
index = 0 size = torch.Size([1, 3, 278770])
39.999996
index = 1 size = torch.Size([1, 3, 278770])
22.999998
index = 2 size = torch.Size([1, 3, 278770])
44.0
index = 3 size = torch.Size([1, 3, 278770])
32.0
index = 4 size = torch.Size([1, 3, 278770])
49.000004
index = 5 size = torch.Size([1, 3, 222530])
40.999992
index = 6 size = torch.Size([1, 3, 278770])
60.000004
index = 7 size = torch.Size([1, 3, 222530])
32.000004
index = 8 size = torch.Size([1, 3, 278770])
41.999996
index = 9 size = torch.Size([1, 3, 278770])
66.0
index = 10 size = torch.Size([1, 3, 278770])
65.99999
index = 11 size = torch.Size([1, 3, 278770])
63.0
index = 12 size = torch.Size([1, 3, 278770])
24.0
index = 13 size = torch.Size([1, 3, 278770])
39.999992
index = 14 size = torch.Size([1, 3, 278770])
36.000004
index = 15 size = torch.Size([1, 3, 278770])
42.0
index = 16 size = torch.Size([1, 3, 278770])
49.99999
index = 17 size = torch.Size([1, 3, 278770])
66.0
index = 18 size = torch.Size([1, 3, 278770]

40.000004
index = 155 size = torch.Size([1, 3, 278770])
5.0
index = 156 size = torch.Size([1, 3, 278770])
64.0
index = 157 size = torch.Size([1, 3, 278770])
120.0
index = 158 size = torch.Size([1, 3, 278770])
70.0
index = 159 size = torch.Size([1, 3, 278770])
60.999996
index = 160 size = torch.Size([1, 3, 278770])
24.000002
index = 161 size = torch.Size([1, 3, 222530])
66.00001
index = 162 size = torch.Size([1, 3, 278770])
0.0
index = 163 size = torch.Size([1, 3, 278770])
62.999996
index = 164 size = torch.Size([1, 3, 278770])
37.999992
index = 165 size = torch.Size([1, 3, 278770])
16.000002
index = 166 size = torch.Size([1, 3, 278770])
1.0
index = 167 size = torch.Size([1, 3, 278770])
16.0
index = 168 size = torch.Size([1, 3, 278770])
21.999994
index = 169 size = torch.Size([1, 3, 278770])
0.0
index = 170 size = torch.Size([1, 3, 278770])
40.0
index = 171 size = torch.Size([1, 3, 278770])
0.0
index = 172 size = torch.Size([1, 3, 278770])
14.000001
index = 173 size = torch.Size([1, 3, 

## Computation Functions

In [11]:
def compute_mae(pd, gt):
    pd, gt = np.array(pd), np.array(gt)
    diff = pd - gt
    mae = np.mean(np.abs(diff))
    return mae


def compute_mse(pd, gt):
    pd, gt = np.array(pd), np.array(gt)
    diff = pd - gt
    mse = np.sqrt(np.mean((diff ** 2)))
    return mse

def compute_relerr(pd, gt):
    pd, gt = np.array(pd), np.array(gt)
    diff = pd - gt
    diff = diff[gt > 0]
    gt = gt[gt > 0]
    if (diff is not None) and (gt is not None):
        rmae = np.mean(np.abs(diff) / gt) * 100
        rmse = np.sqrt(np.mean(diff**2 / gt**2)) * 100
    else:
        rmae = 0
        rmse = 0
    return rmae, rmse


def rsquared(pd, gt):
    """ Return R^2 where x and y are array-like."""
    pd, gt = np.array(pd), np.array(gt)
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(pd, gt)
    return r_value**2


def dense_sample2d(x, sx, stride):
    (h,w) = x.shape[:2]
    #idx_img = np.array([i for i in range(h*w)]).reshape(h,w)
    idx_img = np.zeros((h,w),dtype=float)
    
    th = [i for i in range(0, h-sx+1, stride)]
    tw = [j for j in range(0, w-sx+1, stride)]
    norm_vec = np.zeros(len(th)*len(tw))

    for i in th:
        for j in tw:
            idx_img[i:i+sx,j:j+sx] = idx_img[i:i+sx,j:j+sx]+1

    # plot redundancy map
    import os
    import matplotlib.pyplot as plt
    cmap = plt.cm.get_cmap('hot')
    idx_img = idx_img / (idx_img.max())
    idx_img = cmap(idx_img) * 255.
    plt.figure()
    plt.imshow(idx_img.astype(np.uint8))
    plt.axis('off')
    plt.savefig(os.path.join('redundancy_map.pdf'), bbox_inches='tight', dpi = 300)
    plt.close()
   
    idx_img = 1/idx_img
    idx_img = idx_img/sx/sx
    #line order
    idx = 0
    for i in th:
        for j in tw:
            norm_vec[idx] =idx_img[i:i+sx,j:j+sx].sum()
            idx+=1
    
    return norm_vec


def recover_countmap(pred, image, patch_sz, stride):
    pred = pred.reshape(-1)
    imH, imW = image.shape[2:4]
    cntMap = np.zeros((imH, imW), dtype=float)
    norMap = np.zeros((imH, imW), dtype=float)
    
    H = np.arange(0, imH - patch_sz + 1, stride)
    W = np.arange(0, imW - patch_sz + 1, stride)
    cnt = 0
    for h in H:
        for w in W:
            pixel_cnt = pred[cnt] / patch_sz / patch_sz
            cntMap[h:h+patch_sz, w:w+patch_sz] += pixel_cnt
            norMap[h:h+patch_sz, w:w+patch_sz] += np.ones((patch_sz,patch_sz))
            cnt += 1
    return cntMap / (norMap + 1e-12)

## Building the Model

In [12]:
class Encoder(nn.Module):
    def __init__(self, arc='tasselnetv2'):
        super(Encoder, self).__init__()
        if arc == 'tasselnetv2':
            self.encoder = nn.Sequential(
                nn.Conv2d(3, 16, 3, padding=1, bias=False),
                nn.BatchNorm2d(16),
                nn.ReLU(inplace=True),
                nn.MaxPool2d((2, 2), stride=2),
                nn.Conv2d(16, 32, 3, padding=1, bias=False),
                nn.BatchNorm2d(32),
                nn.ReLU(inplace=True),
                nn.MaxPool2d((2, 2), stride=2),
                nn.Conv2d(32, 64, 3, padding=1, bias=False),
                nn.BatchNorm2d(64),
                nn.ReLU(inplace=True),
                nn.MaxPool2d((2, 2), stride=2),
                nn.Conv2d(64, 128, 3, padding=1, bias=False),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
                nn.Conv2d(128, 128, 3, padding=1, bias=False),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
            )
        else:
            raise NotImplementedError

    def forward(self, x):
        x = self.encoder(x)
        return x

In [13]:
class Counter(nn.Module):
    def __init__(self, arc='tasselnetv2', input_size=64, output_stride=8):
        super(Counter, self).__init__()
        k = int(input_size / 8)
        avg_pool_stride = int(output_stride / 8)

        if arc == 'tasselnetv2':
            self.counter = nn.Sequential(
                nn.Conv2d(128, 128, (k, k), bias=False),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
                nn.Conv2d(128, 128, 1, bias=False),
                nn.BatchNorm2d(128),
                nn.ReLU(inplace=True),
                nn.Conv2d(128, 1, 1)
            )
        else:
            raise NotImplementedError

    def forward(self, x):
        x = self.counter(x)
        return x

In [14]:
class Normalizer:
    @staticmethod
    def cpu_normalizer(x, imh, imw, insz, os):
        # CPU normalization
        bs = x.size()[0]
        normx = np.zeros((imh, imw))
        norm_vec = dense_sample2d(normx, insz, os).astype(np.float32)
        x = x.cpu().detach().numpy().reshape(bs, -1) * norm_vec
        return x
    
    @staticmethod
    def gpu_normalizer(x, imh, imw, insz, os):
        _, _, h, w = x.size()            
        #accm = torch.cuda.FloatTensor(1, insz*insz, h*w).fill_(1)     
        accm = torch.FloatTensor(1, insz*insz, h*w).fill_(1) 
        accm = F.fold(accm, (imh, imw), kernel_size=insz, stride=os)
        accm = 1 / accm
        accm /= insz**2
        accm = F.unfold(accm, kernel_size=insz, stride=os).sum(1).view(1, 1, h, w)
        x *= accm
        return x.squeeze().cpu().detach().numpy()

In [15]:
class CountingModels(nn.Module):
    def __init__(self, arc='tasselnetv2', input_size=64, output_stride=8):
        super(CountingModels, self).__init__()
        self.input_size = input_size
        self.output_stride = output_stride

        self.encoder = Encoder(arc)
        self.counter = Counter(arc, input_size, output_stride)
        if arc == 'tasselnetv2':
          #changed
            self.normalizer = Normalizer.gpu_normalizer
        
        self.weight_init()

    def forward(self, x, is_normalize=True):
        imh, imw = x.size()[2:]
        x = self.encoder(x)
        x = self.counter(x)
        if is_normalize:
            x = self.normalizer(x, imh, imw, self.input_size, self.output_stride)
        return x

    def weight_init(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, std=0.01)
                # nn.init.kaiming_uniform_(
                #         m.weight, 
                #         mode='fan_in', 
                #         nonlinearity='relu'
                #         )
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

In [16]:
from time import time

insz, os = 64, 8
imH, imW = 1080, 1920
#net = CountingModels(arc='tasselnetv2', input_size=insz, output_stride=os).cuda()
net = CountingModels(arc='tasselnetv2', input_size=insz, output_stride=os)
with torch.no_grad():
    net.eval()
    #x = torch.randn(1, 3, imH, imW).cuda()
    x = torch.randn(1, 3, imH, imW)
    y = net(x)
    print(y.shape)    

with torch.no_grad():
    frame_rate = np.zeros((100, 1))
    for i in range(100):
        #x = torch.randn(1, 3, imH, imW).cuda()
        x = torch.randn(1, 3, imH, imW)
        #torch.cuda.synchronize()
        start = time()

        y = net(x)

        #torch.cuda.synchronize()
        end = time()

        running_frame_rate = 1 * float(1 / (end - start))
        frame_rate[i] = running_frame_rate
    print(np.mean(frame_rate))

(128, 233)
0.6917676248407554


In [17]:
y1 = np.argmax(y, axis=1)
y1

array([159, 129,  29, 120, 103, 207,  12,  31, 178, 120, 173,  24,  93,
       166, 113, 186, 193,  94,  52, 158, 232, 126,  84,  50, 115, 215,
       167, 160,  95, 148, 167, 138,  15,  49,  94, 139, 183,  91, 126,
       204, 133,   8, 100, 108,  74,  53, 212,  13, 174,  36, 100,  32,
       196, 147, 118,  88, 155, 215, 232, 206,  78, 100, 212,  28,  59,
        49,  68, 208, 160,  53, 190, 169, 226,  94,  72, 217, 193, 170,
        38,  26, 109, 106, 128, 152,  83, 190,  65,  77,  41, 165,  15,
        52, 107,  87,  39,  77, 173,  33, 169, 204,  73,  11,  36, 148,
       221, 114,  26, 115, 152,  57,  89, 176,  69, 142,  26,  84,  15,
       191, 226,  77,  82, 158,  96,  65,  73, 220, 104,  32], dtype=int64)

In [18]:
plt.switch_backend('agg')

In [22]:
# prevent dataloader deadlock, uncomment if deadlock occurs
cv2.setNumThreads(0)
#cudnn.enabled = True

# constant
IMG_SCALE = 1. / 255
IMG_MEAN = [.3405, .4747, .2418]
IMG_STD = [1, 1, 1]

# system-related parameters
DATA_DIR = 'maize_counting_dataset'
DATASET = 'mtc'
EXP = 'exp_directory'
DATA_LIST = 'maize_counting_dataset/train.txt'
DATA_VAL_LIST = 'maize_counting_dataset/test.txt'

RESTORE_FROM = 'model_best.pth.tar'
SNAPSHOT_DIR = './snapshots'
RESULT_DIR = './results'

# model-related parameters
INPUT_SIZE = 64
OUTPUT_STRIDE = 8
MODEL = 'tasselnetv2'
RESIZE_RATIO = 0.125

# training-related parameters
OPTIMIZER = 'sgd'  # choice in ['sgd', 'adam']
BATCH_SIZE = 9
CROP_SIZE = (256, 256)
LEARNING_RATE = 1e-2
MILESTONES = [200, 400]
MOMENTUM = 0.95
MULT = 1
NUM_EPOCHS = 25
#NUM_EPOCHS = 500
NUM_CPU_WORKERS = 0
PRINT_EVERY = 1
RANDOM_SEED = 6
WEIGHT_DECAY = 5e-4
VAL_EVERY = 1

device = torch.device("cpu")
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# add a new entry here if creating a new data loader
dataset_list = {
    'mtc': MaizeTasselDataset
}

In [23]:
def save_checkpoint(state, snapshot_dir, filename='model_ckpt.pth.tar'):
    torch.save(state, '{}/{}'.format(snapshot_dir, filename))


def plot_learning_curves(net, dir_to_save):
    # plot learning curves
    fig = plt.figure(figsize=(16, 9))
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.plot(net.train_loss['epoch_loss'], label='train loss', color='tab:blue')
    ax1.legend(loc='upper right')
    ax2 = fig.add_subplot(1, 2, 2)
    ax2.plot(net.val_loss['epoch_loss'], label='val mae', color='tab:orange')
    ax2.legend(loc='upper right')
    # ax2.set_ylim((0,50))
    fig.savefig(os.path.join(dir_to_save, 'learning_curves.png'), bbox_inches='tight', dpi=300)
    plt.close()

## Train and Validate Methods

In [24]:
def train(net, train_loader, criterion, optimizer, epoch, args):
    # switch to 'train' mode
    net.train()

    # uncomment the following line if the training images don't have the same size
    #cudnn.benchmark = True

    if BATCH_SIZE == 1:
        for m in net.modules():
            if isinstance(m, nn.BatchNorm2d):
                m.eval()

    running_loss = 0.0
    avg_frame_rate = 0.0
    in_sz = INPUT_SIZE
    os = OUTPUT_STRIDE
    #target_filter = torch.cuda.FloatTensor(1, 1, in_sz, in_sz).fill_(1)
    target_filter = torch.FloatTensor(1, 1, in_sz, in_sz).fill_(1)
    for i, sample in enumerate(train_loader):
        #torch.cuda.synchronize()
        start = time()

        inputs, targets = sample['image'], sample['target']
        #inputs, targets = inputs.cuda(), targets.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = net(inputs, is_normalize=False)
        # generate targets
        targets = F.conv2d(targets, target_filter, stride=os)
        # compute loss
        loss = criterion(outputs, targets)

        # backward + optimize
        loss.backward()
        optimizer.step()
        # collect and print statistics
        running_loss += loss.item()

        #torch.cuda.synchronize()
        end = time()

        running_frame_rate = BATCH_SIZE * float(1 / (end - start))
        avg_frame_rate = (avg_frame_rate * i + running_frame_rate) / (i + 1)
        if i % PRINT_EVERY == PRINT_EVERY - 1:
            print('epoch: %d, train: %d/%d, '
                  'loss: %.5f, frame: %.2fHz/%.2fHz' % (
                      epoch,
                      i + 1,
                      len(train_loader),
                      running_loss / (i + 1),
                      running_frame_rate,
                      avg_frame_rate
                  ))
    net.train_loss['epoch_loss'].append(running_loss / (i + 1))

In [25]:
def validate(net, valset, val_loader, criterion, epoch, args):
    # switch to 'eval' mode
    net.eval()
    cudnn.benchmark = False

    image_list = valset.image_list

    #if args.save_output:
     #   epoch_result_dir = os.path.join(args.result_dir, str(epoch))
      #  if not os.path.exists(epoch_result_dir):
       #     os.makedirs(epoch_result_dir)
        #cmap = plt.cm.get_cmap('jet')
    epoch_result_dir = os.path.join(RESULT_DIR, str(epoch))
    if not os.path.exists(epoch_result_dir):
        os.makedirs(epoch_result_dir)
    cmap = plt.cm.get_cmap('jet')
    
    pd_counts = []
    gt_counts = []
    with torch.no_grad():
        avg_frame_rate = 0.0
        for i, sample in enumerate(val_loader):
            #torch.cuda.synchronize()
            start = time()

            image, gtcount = sample['image'], sample['gtcount']
            #inference
            #output = net(image.cuda(), is_normalize=not args["save_output"])
            output = net(image, is_normalize=not args["save_output"])
            #if args.save_output:
            #    output_save = output
                # normalization
            #    output = Normalizer.gpu_normalizer(output, image.size()[2], image.size()[3], args.input_size,
            #                                       args.output_stride)
            output_save = output
            output = Normalizer.gpu_normalizer(output, image.size()[2], image.size()[3], INPUT_SIZE,
                                                   OUTPUT_STRIDE)
            
            # postprocessing
            output = np.clip(output, 0, None)

            pdcount = output.sum()
            gtcount = float(gtcount.numpy())

            #if args.save_output:
#             if args["save_output"]:
#                 _, image_name = os.path.split(image_list[i])
#                 output_save = np.clip(output_save.squeeze().cpu().numpy(), 0, None)
#                 output_save = recover_countmap(output_save, image, INPUT_SIZE, OUTPUT_STRIDE)
#                 output_save = output_save / (output_save.max() + 1e-12)
#                 output_save = cmap(output_save) * 255.
#                 # image composition
#                 image = valset.images[image_list[i]]
#                 nh, nw = output_save.shape[:2]
#                 image = cv2.resize(image, (nw, nh), interpolation=cv2.INTER_CUBIC)
#                 output_save = 0.5 * image + 0.5 * output_save[:, :, 0:3]

#                 dotimage = valset.dotimages[image_list[i]]

#                 fig = plt.figure()
#                 ax1 = fig.add_subplot(1, 2, 1)
#                 ax1.imshow(dotimage.astype(np.uint8))
#                 ax1.get_xaxis().set_visible(False)
#                 ax1.get_yaxis().set_visible(False)
#                 ax2 = fig.add_subplot(1, 2, 2)
#                 ax2.imshow(output_save.astype(np.uint8))
#                 ax2.get_xaxis().set_visible(False)
#                 ax2.get_yaxis().set_visible(False)
#                 fig.suptitle('manual count=%4.2f, inferred count=%4.2f' % (gtcount, pdcount), fontsize=10)
#                 if DATASET == 'mtc':
#                     plt.tight_layout(rect=[0, 0, 1, 1.4])  # maize tassels counting
#                 plt.savefig(os.path.join(epoch_result_dir, image_name.replace('.jpg', '.png')), bbox_inches='tight',
#                             dpi=300)
#                 plt.close()

            # compute mae and mse
            pd_counts.append(pdcount)
            gt_counts.append(gtcount)
            mae = compute_mae(pd_counts, gt_counts)
            mse = compute_mse(pd_counts, gt_counts)
            rmae, rmse = compute_relerr(pd_counts, gt_counts)

            #torch.cuda.synchronize()
            end = time()

            running_frame_rate = 1 * float(1 / (end - start))
            avg_frame_rate = (avg_frame_rate * i + running_frame_rate) / (i + 1)
            if i % PRINT_EVERY == PRINT_EVERY - 1:
                print(
                    'epoch: {0}, test: {1}/{2}, pre: {3:.2f}, gt:{4:.2f}, me:{5:.2f}, mae: {6:.2f}, mse: {7:.2f}, rmae: {8:.2f}%, rmse: {9:.2f}%, frame: {10:.2f}Hz/{11:.2f}Hz'
                        .format(epoch, i + 1, len(val_loader), pdcount, gtcount, pdcount - gtcount, mae, mse, rmae,
                                rmse, running_frame_rate, avg_frame_rate)
                )
            start = time()
    r2 = rsquared(pd_counts, gt_counts)
    np.save(SNAPSHOT_DIR + '/pd.npy', pd_counts)
    np.save(SNAPSHOT_DIR + '/gt.npy', gt_counts)
    print('epoch: {0}, mae: {1:.2f}, mse: {2:.2f}, rmae: {3:.2f}%, rmse: {4:.2f}%, r2: {5:.4f}'.format(epoch, mae, mse,
                                                                                                       rmae, rmse, r2))
    # write to files        
    with open(os.path.join(SNAPSHOT_DIR, EXP + '.txt'), 'a') as f:
        print(
            'epoch: {0}, mae: {1:.2f}, mse: {2:.2f}, rmae: {3:.2f}%, rmse: {4:.2f}%, r2: {5:.4f}'.format(epoch, mae,
                                                                                                         mse, rmae,
                                                                                                         rmse, r2),
            file=f
        )
    with open(os.path.join(SNAPSHOT_DIR, 'counts.txt'), 'a') as f:
        for pd, gt in zip(pd_counts, gt_counts):
            print(
                '{0} {1}'.format(pd, gt),
                file=f
            )
    # save stats
    net.val_loss['epoch_loss'].append(mae)
    net.measure['mae'].append(mae)
    net.measure['mse'].append(mse)
    net.measure['rmae'].append(rmae)
    net.measure['rmse'].append(rmse)
    net.measure['r2'].append(r2)

In [26]:
args = {
    "image_scale":IMG_SCALE,
    "image_mean":IMG_MEAN,
    "image_std": IMG_STD,
    "data_dir":DATA_DIR,
    "dataset":DATASET,
    "exp":EXP,
    "data_list":DATA_LIST,
    "data_val_list":DATA_VAL_LIST,
    "restore_from":RESTORE_FROM,
    "snapshot_dir":SNAPSHOT_DIR,
    "result_dir":RESULT_DIR,
    "save_output":"store_true",
    "input_size":INPUT_SIZE,
    "output_stride":OUTPUT_STRIDE,
    "resize_ratio":RESIZE_RATIO,
    "model":MODEL,
    "optimizer":OPTIMIZER,
    "batch_size":BATCH_SIZE,
    "milestones":MILESTONES,
    "crop_size":CROP_SIZE,
    "evaluate_only":"store_true",
    "learning_rate":LEARNING_RATE,
    "momentum":MOMENTUM,
    "weight_decay":WEIGHT_DECAY,
    "mult":MULT,
    "num_epochs":NUM_EPOCHS,
    "num_workers":NUM_CPU_WORKERS,
    "print_every":PRINT_EVERY,
    "random_seed":RANDOM_SEED,
    "val_every":VAL_EVERY
}

In [27]:
import os
IMG_MEAN = np.array(IMG_MEAN).reshape((1, 1, 3))
IMG_STD = np.array(IMG_STD).reshape((1, 1, 3))

CROP_SIZE = tuple(CROP_SIZE) if len(CROP_SIZE) > 1 else CROP_SIZE

# seeding for reproducbility
#if torch.cuda.is_available():
#    torch.cuda.manual_seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# instantiate dataset
dataset = dataset_list[DATASET]

SNAPSHOT_DIR = os.path.join(SNAPSHOT_DIR, DATASET.lower(), EXP)
if not os.path.exists(SNAPSHOT_DIR):
    os.makedirs(SNAPSHOT_DIR)

RESULT_DIR = os.path.join(RESULT_DIR, DATASET.lower(), EXP)
if not os.path.exists(RESULT_DIR):
    os.makedirs(RESULT_DIR)

RESTORE_FROM = os.path.join(SNAPSHOT_DIR, RESTORE_FROM)

#arguments = vars(args)
#for item in arguments:
    #print(item, ':\t', arguments[item])

In [28]:
# instantiate network
net = CountingModels(
      arc=MODEL,
      input_size=INPUT_SIZE,
      output_stride=OUTPUT_STRIDE
    )

net = nn.DataParallel(net)
#net.cuda()

# filter parameters
learning_params = [p[1] for p in net.named_parameters()]
pretrained_params = []

# define loss function and optimizer
#criterion = nn.L1Loss(reduction='mean').cuda()
criterion = nn.L1Loss(reduction='mean')

if OPTIMIZER == 'sgd':
    optimizer = torch.optim.SGD(
        [
            {'params': learning_params},
            {'params': pretrained_params, 'lr': LEARNING_RATE / MULT},
        ],
        lr=LEARNING_RATE,
        momentum=MOMENTUM,
        weight_decay=WEIGHT_DECAY
    )
elif OPTIMIZER == 'adam':
    optimizer = torch.optim.Adam(
        [
            {'params': learning_params},
            {'params': pretrained_params, 'lr': LEARNING_RATE / MULT},
        ],
        lr=LEARNING_RATE
    )
else:
    raise NotImplementedError

In [29]:
# restore parameters
start_epoch = 0
net.train_loss = {
    'running_loss': [],
    'epoch_loss': []
}
net.val_loss = {
    'running_loss': [],
    'epoch_loss': []
}
net.measure = {
    'mae': [],
    'mse': [],
    'rmae': [],
    'rmse': [],
    'r2': []
}
if RESTORE_FROM is not None:
    if os.path.isfile(RESTORE_FROM):
        checkpoint = torch.load(RESTORE_FROM)
        net.load_state_dict(checkpoint['state_dict'])
        if 'epoch' in checkpoint:
            start_epoch = checkpoint['epoch']
        if 'optimizer' in checkpoint:
            optimizer.load_state_dict(checkpoint['optimizer'])
        if 'train_loss' in checkpoint:
            net.train_loss = checkpoint['train_loss']
        if 'val_loss' in checkpoint:
            net.val_loss = checkpoint['val_loss']
        if 'measure' in checkpoint:
            net.measure['mae'] = checkpoint['measure']['mae'] if 'mae' in checkpoint['measure'] else []
            net.measure['mse'] = checkpoint['measure']['mse'] if 'mse' in checkpoint['measure'] else []
            net.measure['rmae'] = checkpoint['measure']['rmae'] if 'rmae' in checkpoint['measure'] else []
            net.measure['rmse'] = checkpoint['measure']['rmse'] if 'rmse' in checkpoint['measure'] else []
            net.measure['r2'] = checkpoint['measure']['r2'] if 'r2' in checkpoint['measure'] else []
        print("==> load checkpoint '{}' (epoch {})"
              .format(RESTORE_FROM, start_epoch))
    else:
        with open(os.path.join(SNAPSHOT_DIR, EXP + '.txt'), 'a') as f:
            for item in args:
                print(item, ':\t', args[item], file=f)
        print("==> no checkpoint found at '{}'".format(RESTORE_FROM))

==> no checkpoint found at './snapshots\mtc\exp_directory\model_best.pth.tar'


In [30]:
# define transform
transform_train = [
    RandomCrop(CROP_SIZE),
    RandomFlip(),
    Normalize(
        IMG_SCALE,
        IMG_MEAN,
        IMG_STD
    ),
    ToTensor(),
    ZeroPadding(OUTPUT_STRIDE)
]
transform_val = [
    Normalize(
        IMG_SCALE,
        IMG_MEAN,
        IMG_STD
    ),
    ToTensor(),
    ZeroPadding(OUTPUT_STRIDE)
]
composed_transform_train = transforms.Compose(transform_train)
composed_transform_val = transforms.Compose(transform_val)

# define dataset loader
trainset = dataset(
    data_dir=DATA_DIR,
    data_list=DATA_LIST,
    ratio=RESIZE_RATIO,
    train=True,
    transform=composed_transform_train
)
train_loader = DataLoader(
    trainset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_CPU_WORKERS,
    pin_memory=True,
    drop_last=True
)
valset = dataset(
    data_dir=DATA_DIR,
    data_list=DATA_LIST,
    ratio=RESIZE_RATIO,
    train=False,
    transform=composed_transform_val
)
val_loader = DataLoader(
    valset,
    batch_size=1,
    shuffle=False,
    num_workers=NUM_CPU_WORKERS,
    pin_memory=True
)

## Testing Pytorch Deployment Codes

In [47]:
model = CountingModels(arc='tasselnetv2', input_size=64, output_stride=8)
model = nn.DataParallel(model)
model.load_state_dict(torch.load('density_model.pt'))
model = model.eval()

In [28]:
model = torch.load('snapshots/mtc/exp_directory/whole_model.pt')
model.eval()

DataParallel(
  (module): CountingModels(
    (encoder): Encoder(
      (encoder): Sequential(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
        (4): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (6): ReLU(inplace=True)
        (7): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
        (8): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (10): ReLU(inplace=True)
        (11): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilati

In [40]:
image = read_image('Sample_Images/T0006_XM_20120815160306_01.jpg')
image.shape

(2736, 3648, 3)

In [43]:
#Resize
#img_file = Image.open('Sample_Images/T0006_XM_20120815160306_01.jpg')
image = read_image('Sample_Images/T0006_XM_20120815160306_01.jpg')
annotation = sio.loadmat('Sample_Images/T0006_XM_20120815160306_01.mat')
h, w = image.shape[:2]
nh = int(np.ceil(h * 0.125))
nw = int(np.ceil(w * 0.125))
image = cv2.resize(image, (nw, nh), interpolation = cv2.INTER_CUBIC)
#target = np.zeros((nh, nw), dtype=np.float32)
#dotimage = image.copy()
test_img = image.copy()
image.shape

(342, 456, 3)

In [44]:
#Normalize
image = image.astype('float32')
# pixel normalization
image = (IMG_SCALE * image - IMG_MEAN) / IMG_STD
image = image.astype('float32')

In [45]:
image = image.transpose((2, 0, 1))
image = torch.from_numpy(image)

In [46]:
image.size()

torch.Size([3, 342, 456])

In [47]:
#Zero Padding
psize=32
h,w = image.size()[-2:]
ph,pw = (psize-h%psize),(psize-w%psize)

(pl, pr) = (pw//2, pw-pw//2) if pw != psize else (0, 0)
(pt, pb) = (ph//2, ph-ph//2) if ph != psize else (0, 0)
if (ph!=psize) or (pw!=psize):
    tmp_pad = [pl, pr, pt, pb]
    image = F.pad(image,tmp_pad)

In [49]:
image=image.unsqueeze(0)
image.size()

torch.Size([1, 3, 352, 480])

In [56]:
import io
model.eval()
with torch.no_grad():
    output = model(image, is_normalize=not args["save_output"])
    output_save = output
    output = Normalizer.gpu_normalizer(output, image.size()[2], image.size()[3], INPUT_SIZE,OUTPUT_STRIDE)       
    #postprocessing
    output = np.clip(output, 0, None)
    pdcount = output.sum()
    print(pdcount)
    
    #try density map
    cmap = plt.cm.get_cmap('jet')
    output_save = np.clip(output_save.squeeze().cpu().numpy(), 0, None)
    output_save = recover_countmap(output_save, image, INPUT_SIZE, OUTPUT_STRIDE)
    output_save = output_save / (output_save.max() + 1e-12)
    output_save = cmap(output_save) * 255.
    # image composition
    print(output_save.shape)
    test_img = read_image('Sample_Images/T0006_XM_20120815160306_01.jpg')
    nh, nw = output_save.shape[:2]
    test_img = cv2.resize(test_img, (nw, nh), interpolation=cv2.INTER_CUBIC)
    #output_save = test_img + output_save[:, :, 0:3]
    output_save = 0.5 * test_img + 0.5 * output_save[:, :, 0:3]
    
    fig, ax = plt.subplots(figsize=(15,5))
    ax.imshow(output_save.astype(np.uint8))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    #fig = plt.figure()
    #ax1 = fig.add_subplot(1, 2, 1)
    #ax1.imshow(dotimage.astype(np.uint8))
    #ax1.get_xaxis().set_visible(False)
    #ax1.get_yaxis().set_visible(False)
    #ax2 = fig.add_subplot(1, 2, 2)
    #ax2.imshow(output_save.astype(np.uint8))
    #ax2.get_xaxis().set_visible(False)
    #ax2.get_yaxis().set_visible(False)
    
    #fig.suptitle('inferred count=%4.2f' % (pdcount), fontsize=10)
    
    #plt.tight_layout(rect=[0, 0, 1, 1.4])  # maize tassels counting 
    
    #plt.savefig("test.jpg",bbox_inches='tight',dpi=300)
    
    my_stringIObytes = io.BytesIO()
    print(my_stringIObytes)
    plt.savefig(my_stringIObytes, format='jpg',bbox_inches='tight',dpi=72)
    my_stringIObytes.seek(0)
    my_base64_jpgData = base64.b64encode(my_stringIObytes.read())
    print(my_base64_jpgData)
    
    #plt.show()
    #plt.savefig(os.path.join(epoch_result_dir, image_name.replace('.jpg', '.png')), bbox_inches='tight',dpi=300)

21.263014
(352, 480, 4)
b'...'


In [57]:
valset.images

{}

In [ ]:
image = valset.images[image_list[i]]
nh, nw = output_save.shape[:2]
image = cv2.resize(image, (nw, nh), interpolation=cv2.INTER_CUBIC)

In [84]:
with torch.no_grad():
        for i, sample in enumerate(val_loader):
            image123, gtcount = sample['image'], sample['gtcount']
            print(image123.size())

2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
8 8
torch.Size([1, 3, 288, 432])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
8 8
torch.Size([1, 3, 288, 432])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 1]
torch.Size([1, 3, 344, 456])
2 8
[0, 0, 1, 

In [83]:
image123.size()

torch.Size([1, 3, 344, 456])

## Training the Model

In [31]:
print('alchemy start...')
#if evaluate_only:
#validate(net, valset, val_loader, criterion, start_epoch, args)
    #return

best_mae = 1000000.0
resume_epoch = -1 if start_epoch == 0 else start_epoch
scheduler = MultiStepLR(optimizer, milestones=MILESTONES, gamma=0.1, last_epoch=resume_epoch)
for epoch in range(start_epoch, NUM_EPOCHS):
    # train
    train(net, train_loader, criterion, optimizer, epoch + 1, args)
    if epoch % VAL_EVERY == VAL_EVERY - 1:
        # val
        validate(net, valset, val_loader, criterion, epoch + 1, args)
        # save_checkpoint
        state = {
            'state_dict': net.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': epoch + 1,
            'train_loss': net.train_loss,
            'val_loss': net.val_loss,
            'measure': net.measure
        }
        save_checkpoint(state, SNAPSHOT_DIR, filename='model_ckpt.pth.tar')
        if net.measure['mae'][-1] <= best_mae:
            save_checkpoint(state, SNAPSHOT_DIR, filename='model_best.pth.tar')
            save_checkpoint(state, SNAPSHOT_DIR, filename='model.pth')
            best_mae = net.measure['mae'][-1]
            best_mse = net.measure['mse'][-1]
            best_rmae = net.measure['rmae'][-1]
            best_rmse = net.measure['rmse'][-1]
            best_r2 = net.measure['r2'][-1]
        print(EXP + ' epoch {} finished!'.format(epoch + 1))
        print('best mae: {0:.2f}, best mse: {1:.2f}, best_rmae: {2:.2f}, best_rmse: {3:.2f}, best_r2: {4:.4f}'
              .format(best_mae, best_mse, best_rmae, best_rmse, best_r2))
        plot_learning_curves(net, SNAPSHOT_DIR)
    scheduler.step()

print('Experiments with ' + EXP + ' done!')
with open(os.path.join(SNAPSHOT_DIR, EXP + '.txt'), 'a') as f:
    print(
        'best mae: {0:.2f}, best mse: {1:.2f}, best_rmae: {2:.2f}, best_rmse: {3:.2f}, best_r2: {4:.4f}'
            .format(best_mae, best_mse, best_rmae, best_rmse, best_r2),
        file=f
    )
    print(
        'overall best mae: {0:.2f}, overall best mse: {1:.2f}, overall best_rmae: {2:.2f}, overall best_rmse: {3:.2f}, overall best_r2: {4:.4f}'
            .format(min(net.measure['mae']), min(net.measure['mse']), min(net.measure['rmae']),
                    min(net.measure['rmse']), max(net.measure['r2'])),
        file=f
    )

alchemy start...
57.0
8 8
16.0
8 8
23.0
8 8
49.0
8 8
18.0
8 8
32.0
8 8
65.00001
8 8
14.000002
8 8
67.0
8 8
epoch: 1, train: 1/20, loss: 1.01633, frame: 10.58Hz/10.58Hz
42.000008
8 8
1.0000001
8 8
9.0
8 8
51.000004
8 8
65.99999
8 8
104.00001
8 8
37.000004
8 8
64.00001
8 8
0.0
8 8
epoch: 1, train: 2/20, loss: 0.95981, frame: 15.24Hz/12.91Hz
18.000002
8 8
44.0
8 8
2.0
8 8
85.00001
8 8
17.000002
8 8
38.0
8 8
56.000004
8 8
62.999996
8 8
63.999992
8 8
epoch: 1, train: 3/20, loss: 0.91203, frame: 12.19Hz/12.67Hz
39.999992
8 8
120.0
8 8
69.0
8 8
27.999998
8 8
54.0
8 8
36.000004
8 8
15.999999
8 8
51.0
8 8
19.0
8 8
epoch: 1, train: 4/20, loss: 0.86033, frame: 15.77Hz/13.44Hz
0.0
8 8
68.00001
8 8
38.0
8 8
62.999996
8 8
18.0
8 8
65.99999
8 8
60.0
8 8
70.0
8 8
0.0
8 8
epoch: 1, train: 5/20, loss: 0.83214, frame: 16.05Hz/13.97Hz
62.0
8 8
69.0
8 8
0.0
8 8
30.999998
8 8
41.999996
8 8
1.0
8 8
50.999996
8 8
5.0
8 8
2.9999998
8 8
epoch: 1, train: 6/20, loss: 0.84541, frame: 15.63Hz/14.24Hz
66.0
8 8
54.0


9.0
2 8
[0, 0, 1, 1]
epoch: 1, test: 34/186, pre: 5.22, gt:9.00, me:-3.78, mae: 37.71, mse: 44.46, rmae: 84.66%, rmse: 85.37%, frame: 11.12Hz/10.95Hz
30.999998
2 8
[0, 0, 1, 1]
epoch: 1, test: 35/186, pre: 4.92, gt:31.00, me:-26.08, mae: 37.38, mse: 44.04, rmae: 84.65%, rmse: 85.33%, frame: 10.53Hz/10.94Hz
5.0
2 8
[0, 0, 1, 1]
epoch: 1, test: 36/186, pre: 4.92, gt:5.00, me:-0.08, mae: 36.34, mse: 43.42, rmae: 82.21%, rmse: 84.07%, frame: 11.24Hz/10.95Hz
39.0
2 8
[0, 0, 1, 1]
epoch: 1, test: 37/186, pre: 5.25, gt:39.00, me:-33.75, mae: 36.27, mse: 43.19, rmae: 82.33%, rmse: 84.14%, frame: 9.71Hz/10.91Hz
94.00001
2 8
[0, 0, 1, 1]
epoch: 1, test: 38/186, pre: 5.40, gt:94.00, me:-88.60, mae: 37.65, mse: 44.98, rmae: 82.66%, rmse: 84.43%, frame: 10.32Hz/10.90Hz
37.0
2 8
[0, 0, 1, 1]
epoch: 1, test: 39/186, pre: 4.82, gt:37.00, me:-32.18, mae: 37.51, mse: 44.69, rmae: 82.78%, rmse: 84.50%, frame: 11.37Hz/10.91Hz
68.00001
2 8
[0, 0, 1, 1]
epoch: 1, test: 40/186, pre: 5.09, gt:68.00, me:-62.91

116.99999
2 8
[0, 0, 1, 1]
epoch: 1, test: 88/186, pre: 5.27, gt:117.00, me:-111.73, mae: 38.30, mse: 46.05, rmae: 87.76%, rmse: 96.61%, frame: 10.76Hz/11.12Hz
0.0
2 8
[0, 0, 1, 1]
epoch: 1, test: 89/186, pre: 5.12, gt:0.00, me:5.12, mae: 37.93, mse: 45.79, rmae: 87.76%, rmse: 96.61%, frame: 11.50Hz/11.12Hz
0.0
2 8
[0, 0, 1, 1]
epoch: 1, test: 90/186, pre: 5.04, gt:0.00, me:5.04, mae: 37.56, mse: 45.54, rmae: 87.76%, rmse: 96.61%, frame: 11.24Hz/11.13Hz
62.999996
2 8
[0, 0, 1, 1]
epoch: 1, test: 91/186, pre: 4.91, gt:63.00, me:-58.09, mae: 37.79, mse: 45.70, rmae: 87.82%, rmse: 96.55%, frame: 11.63Hz/11.13Hz
2.0
2 8
[0, 0, 1, 1]
epoch: 1, test: 92/186, pre: 5.02, gt:2.00, me:3.02, mae: 37.41, mse: 45.45, rmae: 88.58%, rmse: 97.39%, frame: 11.12Hz/11.13Hz
1.9999998
8 8
epoch: 1, test: 93/186, pre: 3.93, gt:2.00, me:1.93, mae: 37.03, mse: 45.21, rmae: 88.68%, rmse: 97.39%, frame: 14.71Hz/11.17Hz
62.000004
2 8
[0, 0, 1, 1]
epoch: 1, test: 94/186, pre: 5.24, gt:62.00, me:-56.76, mae: 37.24

50.999996
2 8
[0, 0, 1, 1]
epoch: 1, test: 141/186, pre: 4.93, gt:51.00, me:-46.07, mae: 36.21, mse: 43.81, rmae: 91.71%, rmse: 105.37%, frame: 10.11Hz/10.96Hz
96.0
2 8
[0, 0, 1, 1]
epoch: 1, test: 142/186, pre: 5.32, gt:96.00, me:-90.68, mae: 36.59, mse: 44.32, rmae: 91.73%, rmse: 105.29%, frame: 10.88Hz/10.96Hz
17.999998
2 8
[0, 0, 1, 1]
epoch: 1, test: 143/186, pre: 4.95, gt:18.00, me:-13.05, mae: 36.43, mse: 44.17, rmae: 91.58%, rmse: 105.08%, frame: 11.50Hz/10.96Hz
99.99999
2 8
[0, 0, 1, 1]
epoch: 1, test: 144/186, pre: 5.23, gt:100.00, me:-94.77, mae: 36.83, mse: 44.72, rmae: 91.61%, rmse: 105.01%, frame: 10.88Hz/10.96Hz
63.999996
2 8
[0, 0, 1, 1]
epoch: 1, test: 145/186, pre: 5.03, gt:64.00, me:-58.97, mae: 36.99, mse: 44.84, rmae: 91.61%, rmse: 104.92%, frame: 11.91Hz/10.97Hz
48.0
2 8
[0, 0, 1, 1]
epoch: 1, test: 146/186, pre: 5.10, gt:48.00, me:-42.90, mae: 37.03, mse: 44.82, rmae: 91.60%, rmse: 104.81%, frame: 10.11Hz/10.96Hz
5.0000005
2 8
[0, 0, 1, 1]
epoch: 1, test: 147/186

epoch: 2, train: 10/20, loss: 0.63917, frame: 17.25Hz/16.98Hz
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
epoch: 2, train: 11/20, loss: 0.63457, frame: 17.15Hz/17.00Hz
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
epoch: 2, train: 12/20, loss: 0.63959, frame: 17.28Hz/17.02Hz
70.0
8 8
8 8
8 8
8 8
8 8
8 8
30.999996
8 8
8 8
8 8
epoch: 2, train: 13/20, loss: 0.63152, frame: 16.71Hz/17.00Hz
8 8
8 8
8 8
39.0
8 8
8 8
8 8
33.999996
8 8
8 8
8 8
epoch: 2, train: 14/20, loss: 0.62158, frame: 15.91Hz/16.92Hz
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
epoch: 2, train: 15/20, loss: 0.62240, frame: 15.37Hz/16.82Hz
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
epoch: 2, train: 16/20, loss: 0.62323, frame: 16.55Hz/16.80Hz
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
epoch: 2, train: 17/20, loss: 0.62388, frame: 16.61Hz/16.79Hz
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
epoch: 2, train: 18/20, loss: 0.63299, frame: 17.15Hz/16.81Hz
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
epoch: 2, train: 19/20, loss: 0.62668, frame: 16.96Hz/16.82Hz
8 8
8 8
8 8
8 8
8 8
8 8
8 

...

2 8
[0, 0, 1, 1]
epoch: 24, test: 135/186, pre: 31.99, gt:45.00, me:-13.01, mae: 10.46, mse: 13.17, rmae: 37.65%, rmse: 62.54%, frame: 10.88Hz/10.98Hz
2 8
[0, 0, 1, 1]
epoch: 24, test: 136/186, pre: 31.08, gt:18.00, me:13.08, mae: 10.48, mse: 13.17, rmae: 37.92%, rmse: 62.63%, frame: 10.88Hz/10.98Hz
2 8
[0, 0, 1, 1]
epoch: 24, test: 137/186, pre: 77.72, gt:85.00, me:-7.28, mae: 10.46, mse: 13.13, rmae: 37.69%, rmse: 62.39%, frame: 11.24Hz/10.99Hz
2 8
[0, 0, 1, 1]
epoch: 24, test: 138/186, pre: 27.38, gt:18.00, me:9.38, mae: 10.45, mse: 13.11, rmae: 37.81%, rmse: 62.31%, frame: 11.12Hz/10.99Hz
2 8
[0, 0, 1, 1]
epoch: 24, test: 139/186, pre: 25.83, gt:44.00, me:-18.17, mae: 10.51, mse: 13.15, rmae: 37.83%, rmse: 62.18%, frame: 11.50Hz/10.99Hz
2 8
[0, 0, 1, 1]
epoch: 24, test: 140/186, pre: 57.48, gt:62.00, me:-4.52, mae: 10.46, mse: 13.11, rmae: 37.60%, rmse: 61.94%, frame: 11.24Hz/10.99Hz
2 8
[0, 0, 1, 1]
epoch: 24, test: 141/186, pre: 31.70, gt:51.00, me:-19.30, mae: 10.53, mse: 13.17,

epoch: 25, train: 4/20, loss: 0.35874, frame: 17.19Hz/17.45Hz
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
epoch: 25, train: 5/20, loss: 0.35593, frame: 17.59Hz/17.48Hz
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
epoch: 25, train: 6/20, loss: 0.33981, frame: 17.38Hz/17.46Hz
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
epoch: 25, train: 7/20, loss: 0.34398, frame: 13.84Hz/16.95Hz
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
epoch: 25, train: 8/20, loss: 0.33899, frame: 16.15Hz/16.85Hz
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
epoch: 25, train: 9/20, loss: 0.32887, frame: 16.52Hz/16.81Hz
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
epoch: 25, train: 10/20, loss: 0.33079, frame: 17.20Hz/16.85Hz
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
epoch: 25, train: 11/20, loss: 0.33018, frame: 16.93Hz/16.86Hz
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
epoch: 25, train: 12/20, loss: 0.33794, frame: 17.38Hz/16.90Hz
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
epoch: 25, train: 13/20, loss: 0.33643, frame: 17.25Hz/16.93Hz
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
8 8
epoch: 25, train

2 8
[0, 0, 1, 1]
epoch: 25, test: 47/186, pre: 56.54, gt:63.00, me:-6.46, mae: 6.67, mse: 9.77, rmae: 22.64%, rmse: 40.03%, frame: 10.76Hz/11.03Hz
8 8
epoch: 25, test: 48/186, pre: 29.50, gt:4.00, me:25.50, mae: 7.06, mse: 10.35, rmae: 36.62%, rmse: 103.95%, frame: 14.09Hz/11.09Hz
2 8
[0, 0, 1, 1]
epoch: 25, test: 49/186, pre: 49.04, gt:51.00, me:-1.96, mae: 6.95, mse: 10.24, rmae: 35.89%, rmse: 102.79%, frame: 11.24Hz/11.10Hz
2 8
[0, 0, 1, 1]
epoch: 25, test: 50/186, pre: 29.30, gt:37.00, me:-7.70, mae: 6.97, mse: 10.20, rmae: 35.56%, rmse: 101.71%, frame: 11.00Hz/11.09Hz
2 8
[0, 0, 1, 1]
epoch: 25, test: 51/186, pre: 0.16, gt:0.00, me:0.16, mae: 6.84, mse: 10.10, rmae: 35.56%, rmse: 101.71%, frame: 10.64Hz/11.09Hz
2 8
[0, 0, 1, 1]
epoch: 25, test: 52/186, pre: 56.53, gt:62.00, me:-5.47, mae: 6.81, mse: 10.03, rmae: 35.00%, rmse: 100.63%, frame: 11.50Hz/11.09Hz
2 8
[0, 0, 1, 1]
epoch: 25, test: 53/186, pre: 0.24, gt:0.00, me:0.24, mae: 6.69, mse: 9.94, rmae: 35.00%, rmse: 100.63%, fra

2 8
[0, 0, 1, 1]
epoch: 25, test: 104/186, pre: 30.77, gt:44.00, me:-13.23, mae: 6.55, mse: 9.28, rmae: 27.47%, rmse: 73.46%, frame: 11.12Hz/11.15Hz
2 8
[0, 0, 1, 1]
epoch: 25, test: 105/186, pre: 62.28, gt:54.00, me:8.28, mae: 6.57, mse: 9.27, rmae: 27.35%, rmse: 73.10%, frame: 10.42Hz/11.14Hz
2 8
[0, 0, 1, 1]
epoch: 25, test: 106/186, pre: 27.30, gt:19.00, me:8.30, mae: 6.58, mse: 9.26, rmae: 27.52%, rmse: 72.85%, frame: 11.37Hz/11.14Hz
2 8
[0, 0, 1, 1]
epoch: 25, test: 107/186, pre: 26.57, gt:17.00, me:9.57, mae: 6.61, mse: 9.26, rmae: 27.81%, rmse: 72.70%, frame: 11.24Hz/11.15Hz
2 8
[0, 0, 1, 1]
epoch: 25, test: 108/186, pre: 77.12, gt:68.00, me:9.12, mae: 6.63, mse: 9.26, rmae: 27.66%, rmse: 72.35%, frame: 10.76Hz/11.14Hz
2 8
[0, 0, 1, 1]
epoch: 25, test: 109/186, pre: 18.39, gt:19.00, me:-0.61, mae: 6.58, mse: 9.22, rmae: 27.42%, rmse: 71.99%, frame: 11.00Hz/11.14Hz
2 8
[0, 0, 1, 1]
epoch: 25, test: 110/186, pre: 10.27, gt:9.00, me:1.27, mae: 6.53, mse: 9.18, rmae: 27.29%, rmse: 

2 8
[0, 0, 1, 1]
epoch: 25, test: 160/186, pre: 50.88, gt:70.00, me:-19.12, mae: 6.21, mse: 8.80, rmae: 28.47%, rmse: 75.67%, frame: 11.12Hz/11.03Hz
2 8
[0, 0, 1, 1]
epoch: 25, test: 161/186, pre: 65.97, gt:61.00, me:4.97, mae: 6.20, mse: 8.78, rmae: 28.33%, rmse: 75.42%, frame: 11.12Hz/11.03Hz
8 8
epoch: 25, test: 162/186, pre: 22.93, gt:24.00, me:-1.07, mae: 6.17, mse: 8.76, rmae: 28.18%, rmse: 75.17%, frame: 13.52Hz/11.05Hz
2 8
[0, 0, 1, 1]
epoch: 25, test: 163/186, pre: 77.38, gt:66.00, me:11.38, mae: 6.20, mse: 8.78, rmae: 28.10%, rmse: 74.94%, frame: 11.37Hz/11.05Hz
2 8
[0, 0, 1, 1]
epoch: 25, test: 164/186, pre: 0.96, gt:0.00, me:0.96, mae: 6.17, mse: 8.75, rmae: 28.10%, rmse: 74.94%, frame: 10.11Hz/11.05Hz
2 8
[0, 0, 1, 1]
epoch: 25, test: 165/186, pre: 62.84, gt:63.00, me:-0.16, mae: 6.14, mse: 8.72, rmae: 27.92%, rmse: 74.69%, frame: 10.88Hz/11.04Hz
2 8
[0, 0, 1, 1]
epoch: 25, test: 166/186, pre: 49.39, gt:38.00, me:11.39, mae: 6.17, mse: 8.74, rmae: 27.94%, rmse: 74.49%, fra

In [32]:
net_scripted = torch.jit.script(net) # Export to TorchScript
net_scripted.save('density_model.pt') # Save

NotSupportedError: Compiled functions can't take variable number of arguments or use keyword-only arguments with defaults:
  File "D:\anaconda3\lib\site-packages\torch\nn\parallel\data_parallel.py", line 147
    def forward(self, *inputs, **kwargs):
                                ~~~~~~~ <--- HERE
        with torch.autograd.profiler.record_function("DataParallel.forward"):
            if not self.device_ids:


In [33]:
torch.save(net.state_dict(), "density_model.pt")

In [31]:
torch.save(net, 'snapshots/mtc/exp_directory/whole_model.pt')

## Validate the Model

In [32]:
validate(net, valset, val_loader, criterion, start_epoch, args)

2 8
[0, 0, 1, 1]
epoch: 3, test: 1/186, pre: 118.44, gt:97.00, me:21.44, mae: 21.44, mse: 21.44, rmae: 22.10%, rmse: 22.10%, frame: 1.00Hz/1.00Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 2/186, pre: 38.28, gt:40.00, me:-1.72, mae: 11.58, mse: 15.21, rmae: 13.20%, rmse: 15.92%, frame: 1.03Hz/1.01Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 3/186, pre: 25.41, gt:23.00, me:2.41, mae: 8.52, mse: 12.49, rmae: 12.30%, rmse: 14.34%, frame: 1.01Hz/1.01Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 4/186, pre: 57.43, gt:44.00, me:13.43, mae: 9.75, mse: 12.73, rmae: 16.85%, rmse: 19.68%, frame: 1.03Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 5/186, pre: 29.10, gt:32.00, me:-2.90, mae: 8.38, mse: 11.46, rmae: 15.29%, rmse: 18.06%, frame: 1.02Hz/1.02Hz
8 8
epoch: 3, test: 6/186, pre: 6.51, gt:49.00, me:-42.49, mae: 14.07, mse: 20.26, rmae: 27.20%, rmse: 39.05%, frame: 1.13Hz/1.04Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 7/186, pre: 52.86, gt:41.00, me:11.86, mae: 13.75, mse: 19.29, rmae: 27.45%, rmse: 37.77%, frame: 1.05Hz/1.04Hz
8

epoch: 3, test: 58/186, pre: 78.10, gt:65.00, me:13.10, mae: 10.41, mse: 14.14, rmae: 40.29%, rmse: 102.71%, frame: 1.04Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 59/186, pre: 25.48, gt:21.00, me:4.48, mae: 10.31, mse: 14.03, rmae: 39.94%, rmse: 101.78%, frame: 1.00Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 60/186, pre: 63.69, gt:52.00, me:11.69, mae: 10.34, mse: 13.99, rmae: 39.61%, rmse: 100.88%, frame: 1.02Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 61/186, pre: 51.28, gt:51.00, me:0.28, mae: 10.17, mse: 13.88, rmae: 38.90%, rmse: 99.96%, frame: 1.04Hz/1.02Hz
8 8
epoch: 3, test: 62/186, pre: 31.81, gt:36.00, me:-4.19, mae: 10.08, mse: 13.78, rmae: 38.42%, rmse: 99.08%, frame: 1.21Hz/1.03Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 63/186, pre: 1.53, gt:0.00, me:1.53, mae: 9.94, mse: 13.67, rmae: 38.42%, rmse: 99.08%, frame: 1.05Hz/1.03Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 64/186, pre: 1.66, gt:1.00, me:0.66, mae: 9.79, mse: 13.56, rmae: 38.89%, rmse: 98.59%, frame: 1.00Hz/1.03Hz
2 8
[0, 0, 1, 1

epoch: 3, test: 115/186, pre: 88.08, gt:56.00, me:32.08, mae: 10.13, mse: 13.58, rmae: 33.31%, rmse: 75.62%, frame: 1.03Hz/1.03Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 116/186, pre: 14.37, gt:15.00, me:-0.63, mae: 10.05, mse: 13.52, rmae: 33.03%, rmse: 75.26%, frame: 0.91Hz/1.03Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 117/186, pre: 65.72, gt:52.00, me:13.72, mae: 10.08, mse: 13.52, rmae: 32.97%, rmse: 74.95%, frame: 1.04Hz/1.03Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 118/186, pre: 79.73, gt:47.00, me:32.73, mae: 10.27, mse: 13.80, rmae: 33.31%, rmse: 74.90%, frame: 1.02Hz/1.03Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 119/186, pre: 52.68, gt:51.00, me:1.68, mae: 10.20, mse: 13.74, rmae: 33.03%, rmse: 74.56%, frame: 1.01Hz/1.03Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 120/186, pre: 41.29, gt:41.00, me:0.29, mae: 10.12, mse: 13.69, rmae: 32.74%, rmse: 74.22%, frame: 1.02Hz/1.03Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 121/186, pre: 50.05, gt:45.00, me:5.05, mae: 10.07, mse: 13.64, rmae: 32.55%, rmse: 73.89%, frame: 1.01H

epoch: 3, test: 172/186, pre: 73.31, gt:40.00, me:33.31, mae: 10.50, mse: 14.17, rmae: 34.92%, rmse: 74.40%, frame: 1.03Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 173/186, pre: 0.96, gt:0.00, me:0.96, mae: 10.45, mse: 14.13, rmae: 34.92%, rmse: 74.40%, frame: 0.99Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 174/186, pre: 10.47, gt:14.00, me:-3.53, mae: 10.41, mse: 14.09, rmae: 34.86%, rmse: 74.20%, frame: 1.02Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 175/186, pre: 57.51, gt:54.00, me:3.51, mae: 10.37, mse: 14.06, rmae: 34.68%, rmse: 73.97%, frame: 1.01Hz/1.02Hz
8 8
epoch: 3, test: 176/186, pre: 45.53, gt:64.00, me:-18.47, mae: 10.42, mse: 14.08, rmae: 34.65%, rmse: 73.77%, frame: 1.11Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 177/186, pre: 37.79, gt:41.00, me:-3.21, mae: 10.38, mse: 14.05, rmae: 34.48%, rmse: 73.55%, frame: 1.00Hz/1.02Hz
2 8
[0, 0, 1, 1]
epoch: 3, test: 178/186, pre: 48.05, gt:46.00, me:2.05, mae: 10.33, mse: 14.01, rmae: 34.30%, rmse: 73.33%, frame: 1.01Hz/1.02Hz
2 8
[